In [1]:
# Устанавливаем/меняем текущую папку
import os
os.chdir('/root/jupyterlab/balance')

# Импортируем модуль получения данных
from src.AlfaScraper import AlfaScraper

# Для работы с SQL
from sqlalchemy import create_engine
# Для работы с Postgre
import psycopg2

In [2]:
# Получаем переменные окружения
PG_HOST = os.environ['PG_HOST']
PG_PASSWORD = os.environ['PG_PASSWORD']
LOGIN_NAME = os.environ['LOGIN_NAME']
ALFA_USER = os.environ['ALFA_USER'] 
ALFA_PASSWORD = os.environ['ALFA_PASSWORD'] 

In [3]:
# Создаем подключение к pg
engine = create_engine(f'postgres://{LOGIN_NAME}:{PG_PASSWORD}@{PG_HOST}:5432/{LOGIN_NAME}')
conn = psycopg2.connect(f"host='{PG_HOST}' dbname='{LOGIN_NAME}' user='{LOGIN_NAME}' password='{PG_PASSWORD}'")

In [4]:
# Создаем объект класса            
alfa_scraper = AlfaScraper( 
    alfa_login = ALFA_USER
    ,alfa_password = ALFA_PASSWORD
    ,engine = engine
    ,conn = conn
    ,host = PG_HOST
)

In [51]:
# Получаем данные с Альфа-Банка
alfa_scraper.data()

Получаем и записываем баланс, получаем файл выписки
Получаем и записываем выписку
Закрываем браузер


In [6]:
# Закрываем браузер
# alfa_scraper.close_all()

In [ ]:
# Сохраняем скрин
alfa_scraper.driver.save_screenshot('/share/screenshot.png')

In [7]:
# Получаем данные с DWH
df = alfa_scraper.account()
df.head()

In [8]:
# Расходы 2021 по категориям
df[(df['type'].str.contains('Расход')) & (df['date'] > '2021-01-01')].groupby('category')['sum'].sum().sort_values(ascending = False) / df[(df['type'].str.contains('Расход')) & (df['date'] > '2021-01-01')]['sum'].sum()

In [9]:
# Расходы 2021 по назначению 
df[(df['type'].str.contains('Расход')) & (df['date'] > '2021-01-01')].groupby('description')['sum'].sum().sort_values(ascending = False) / df[(df['type'].str.contains('Расход')) & (df['date'] > '2021-01-01')]['sum'].sum()

In [10]:
# Расходы по дням
costs_daily_df = df[(df['type'].str.contains('Расход')) & (df['date'] > '2021-01-01')].groupby('date', as_index = False)['sum'].sum()
fig = costs_daily_df.sort_values(by = 'date').plot.line(
    x='date'
    ,y='sum'
    ,title="Расходы по дням"
)
fig.update_xaxes(title_text='День')
fig.update_yaxes(title_text='Расходы, руб.')
fig.show()